In [1]:
import change_path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ucscgenome
import pyliftover
from tqdm import tqdm

#load in the DNA
genome19 = ucscgenome.Genome('/home/kal/.ucscgenome/hg19.2bit')
genome38 = ucscgenome.Genome('/home/kal/.ucscgenome/hg38.2bit')
lo = pyliftover.LiftOver('/home/kal/K27act_models/eqtl_data/hg38ToHg19.over.chain')

['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/home/kal/VENV/lib/python3.5/site-packages/', '/home/kal/VENV/lib64/python3.5/site-packages/', '/home/kal/VENV/lib/python3.5/site-packages/IPython/extensions/', '/home/kal/K27act_models/bin/', '/home/kal/TF_models/bin/', '/home/kal/.ipython']


In [ ]:
# import the ibd varients
ibd_l = pd.read_table('/home/kal/K27act_models/eqtl_data/IBD_list_of_varients_raw.csv')
print(len(ibd_l))
ibd_l.head()

In [ ]:
# check if reference matches genome
count=0
ibd_l['good'] = np.nan
for idx, row in ibd_l.iterrows():
    if genome[row.chr][row.position].lower() != row.refAllele[0].lower():
        print('Should be {}'.format(genome[row.chr][row.position].lower()))
        print('Actually is {}'.format(row.refAllele.lower()))
        print('Alternate is {}'.format(row.altAllele.lower()))
        count+=1
        if row.refAllele.lower() == 'i':
            ibd_l.at[idx, 'good'] = True
        else:
            ibd_l.at[idx, 'good'] = False
    else:
        ibd_l.at[idx, 'good'] = True

        
print(count)

In [ ]:
filter = [False if len(a)>256 else True for a in ibd_l.refAllele]
print(sum(filter))
print(len(ibd_l))

ibd_l = ibd_l[filter]
ibd_l = ibd_l[ibd_l.good == 1]

print(len(ibd_l))

In [ ]:
ibd_l.to_csv('/home/kal/K27act_models/eqtl_data/IBD_list_of_varients.csv', sep='\t', index=False)

In [ ]:
# import the other ibd varients
ibd_c = pd.read_table('/home/kal/K27act_models/eqtl_data/IBD_credible_sets_raw.csv')
ibd_c.chr = ['chr' + str(n) for n in ibd_c['chr']]
ibd_c = ibd_c.rename(index=str, columns={'A0':'refAllele', 'A1': 'altAllele', 'position.lead':'position'})
ibd_c.position = ibd_c.position - 1
ibd_c = ibd_c.fillna(False)
print(len(ibd_c))
ibd_c.head()

In [ ]:
# check if reference matches genome
count=0
for idx, row in ibd_c.iterrows():
    if genome[row.chr][row.position].lower() != row.refAllele[0].lower():
        print('Should be {}'.format(genome[row.chr][row.position].lower()))
        print('Actually is {}'.format(row.refAllele.lower()))
        print('Alternate is {}'.format(row.altAllele.lower()))
        count+=1
        if row.refAllele.lower() == 'i':
            ibd_c.at[idx, 'good'] = True
        else:
            ibd_c.at[idx, 'good'] = False
    else:
        ibd_c.at[idx, 'good'] = True
print(count)
ibd_c = ibd_c[ibd_c.good == 1]

In [ ]:
ibd_c.to_csv('/home/kal/K27act_models/eqtl_data/IBD_credible_sets.csv', sep='\t', index=False)

In [2]:
# import the eqtl data
eqtl = pd.read_table('/home/kal/K27act_models/eqtl_data/Cells_EBV-transformed_lymphocytes.hg19.csv')
eqtl = eqtl.rename(index=str, columns={'eqtl.ref.allele':'refAllele', 'eqtl.alt.allele': 'altAllele'})
eqtl.position = eqtl.position - 1
eqtl['position38'] = [int(e.split('_')[1]) - 1 for e in eqtl['variant_id']]
eqtl['chr38'] = [e.split('_')[0] for e in eqtl['variant_id']]
eqtl.head()

,chr,position,variant_id,gene_id,variant,prob.in.set,PosteriorProb,gene_name,CredibleSetSize,CredibleSet,...,ma_count,maf,pval_nominal,slope,slope_se,refAllele,altAllele,eqtl_name,position38,chr38
0,chr10,102087037,chr10_100327281_T_C_b38,ENSG00000224934.2,102087038,0.019453,0.023665,RP11-441O15.3,27.0,ENSG00000224934.2,...,36,0.122449,0.000652,0.375461,0.107300,T,C,chr10:102087038:RP11-441O15.3,100327280,chr10
1,chr10,102087112,chr10_100327356_G_T_b38,ENSG00000224934.2,102087113,0.005739,0.006981,RP11-441O15.3,NaN,NaN,...,58,0.197279,0.001981,-0.268628,0.084970,G,T,chr10:102087113:RP11-441O15.3,100327355,chr10
2,chr10,102103036,chr10_100343280_C_T_b38,ENSG00000119929.12,102103037,0.051836,0.047800,CUTC,43.0,ENSG00000119929.12,...,33,0.112245,0.000046,0.452428,0.107065,C,T,chr10:102103037:CUTC,100343279,chr10
3,chr10,102105629,chr10_100345873_T_C_b38,ENSG00000119929.12,102105630,0.441053,0.406717,CUTC,43.0,ENSG00000119929.12,...,32,0.108844,0.000003,0.502150,0.102816,T,C,chr10:102105630:CUTC,100345872,chr10
4,chr10,102127491,chr10_100367735_A_G_b38,ENSG00000119929.12,102127492,0.043167,0.039806,CUTC,43.0,ENSG00000119929.12,...,40,0.136054,0.000060,-0.380238,0.091461,A,G,chr10:102127492:CUTC,100367734,chr10


In [3]:
# check if reference matches genome
count=0
good=list()
for idx, row in tqdm(eqtl.iterrows(), total=len(eqtl)):
    if genome19[row.chr][row.position].lower() != row.refAllele[0].lower():
        #print('Should be {}'.format(genome19[row.chr][row.position].lower()))
        #print('Actually is {}'.format(row.refAllele.lower()))
        #print('Alternate is {}'.format(row.altAllele.lower()))
        good.append(False)
    else:
        good.append(True)
        count+=1
        
print(count)
print(len(eqtl))
print(count/len(eqtl)*100)

100%|██████████| 239048/239048 [00:24<00:00, 9822.58it/s] 

235686
239048
98.59358789866471


In [4]:
eqtl['matches'] = good

In [ ]:
# map from hg38 to hg19
good_scores = list()
bad_scores=list()
verb=False
# pyliftover produces tuples (target_chromosome, target_position, target_strand, conversion_chain_score),
for index, row in tqdm(eqtl.iterrows(), total=len(eqtl)):
    new_cords = lo.convert_coordinate(row.chr38, row.position38)
    if len(new_cords) > 1 and verb:
        print(new_cords)
    eqtl.at[idx, 'chr19'] = new_cords[0][0]
    eqtl.at[idx, 'position19'] = new_cords[0][1]
    if row.refAllele.lower()[0] != genome[new_cords[0][0]][new_cords[0][1]].lower():
        if verb:
            print('old' + str([row.chr38, row.position38]))
            print('new' + str(new_cords))
            print('38 is {}, reference is {}, 19 is {}'.format(genome38[row.chr38][row.position38].lower(), 
                                                        row.refAllele.lower(), 
                                                       genome[new_cords[0][0]][new_cords[0][1]].lower()))
        bad_scores.append(new_cords[0][3])
    else:
        good_scores.append(new_cords[0][3])

In [ ]:
plt.hist(bad_scores, label='Negatives', bins=10)  
plt.hist(good_scores, label='Postitives', bins=10) 
plt.legend()
plt.title('Score certainty for SNVs mapped hg38 to hg 19')
plt.show()

print(len(bad_scores))

In [ ]:
# write out hg 38 eqtl data
eqtl38['position'] = eqtl['position38']
eqtl38['chr'] = eqtl['chr38']
columns = ['chr', 'position', 'variant_id', 'refAllele', 'altAllele', 'gene_id', 'variant', 'prob.in.set',
           'PosteriorProb', 'gene_name', 'CredibleSetSize', 'CredibleSet', 'CredibleSetName', 'median.expression', 
           'missingness', 'mappability.k50', 'tss_distance', 'ma_samples', 'ma_count', 'maf', 'pval_nominal', 
           'slope', 'slope_se', 'eqtl_name']
eqtl38.to_csv('/home/kal/K27act_models/eqtl_data/Cells_EBV-transformed_lymphocytes.hg38.csv', index=None, sep='\t', columns=columns)

In [6]:
# write out hg 19 eqtl data
eqtl19 = eqtl[eqtl['matches']]
columns = ['chr', 'position', 'variant_id', 'refAllele', 'altAllele', 'gene_id', 'variant', 'prob.in.set',
           'PosteriorProb', 'gene_name', 'CredibleSetSize', 'CredibleSet', 'CredibleSetName', 'median.expression', 
           'missingness', 'mappability.k50', 'tss_distance', 'ma_samples', 'ma_count', 'maf', 'pval_nominal', 
           'slope', 'slope_se', 'eqtl_name']
eqtl19.to_csv('/home/kal/K27act_models/eqtl_data/Cells_EBV-transformed_lymphocytes.hg19.csv', index=None, sep='\t', columns=columns)